In [43]:
"""
To demonstrate the spectral models in spectra.py.
"""
import os
import sys
import pickle
import matplotlib
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from importlib import reload
from ipywidgets import interact, interactive, fixed, interact_manual
from astropy.wcs import WCS
from astropy.io import fits

import spectra
reload(spectra)

# change default matplotlib fonts
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['axes.labelsize'] = 32
plt.rcParams['legend.fontsize'] = 26

## Demonstration of spectral models

In [44]:
# parameters
T_bg = 2.73 # [K]
v = np.linspace(-30, 30, 1000)  # [km/s]
model = 'NH3_1-1'
para_type = 'T-N'
'''
C17O_1-0
C17O_2-1
C18O_1-0
CO_2-1
H13CO+_1-0
N2H+_1-0
NH2D_1-1
NH3_x-x, x=1-9   
'''
Switch = {'thin': False, 'intensity': True, 'hyperfine': False, 
          'collapsed_hyperfine': False}
Switch_thin = {'thin': True, 'intensity': True, 'hyperfine': True, 
               'collapsed_hyperfine': False}

def f(draw_thin, draw_hyper, T_ex, lgN, sigma_v):
    para = [T_ex, lgN, 0, sigma_v]
    y = spectra.specmod(v, model, T_bg, Switch, para_type, *para)
    y_thin = spectra.specmod(v, model, T_bg, Switch_thin, para_type, *para)

    plt.figure(figsize=(20,8))
    # plt.xlim(-7,7)
    plt.plot(v, y, linewidth=3, c='k', label='Actual')
    if draw_thin:
        plt.plot(v, y_thin[0], c='k', linewidth=1, label='Opatically thin')
    if draw_hyper:
        for i in range(1,len(y_thin)):
            plt.plot(v, y_thin[i], linewidth=1)
    plt.text(.05,.9,model,fontsize=30,transform=plt.gca().transAxes)
    plt.legend()
    plt.grid()
    plt.xlabel('v_lsr (km/s)')
    plt.ylabel('Intensity (Jy/sr)')

interact(f, T_ex=(3.75,200,.1), lgN=(13.12,18.,.01), sigma_v=(.09,3,.01),
         draw_thin=True, draw_hyper=True)

interactive(children=(Checkbox(value=True, description='draw_thin'), Checkbox(value=True, description='draw_hy…

<function __main__.f(draw_thin, draw_hyper, T_ex, lgN, sigma_v)>

## Demonstration of NH3 inversion lines

In [48]:
# parameters
T_bg = 2.73 # [K]
Switch = {'thin': False, 'intensity': True, 'hyperfine': False, 
          'collapsed_hyperfine': False}
# C = ['r','orangered','orange','y','green','lime','cyan','blue','purple']
C = ['r','orange','green','blue']
v = np.linspace(-40,40,300)
para_type = 'T-N'


def f(T_ex,lgN,sigma_v):
    para = [T_ex,lgN,0,sigma_v] 
    
    plt.figure(figsize=(25,10))
    plt.subplot(121)
    # plt.ylim(0,5e8)
    
    for i in range(1,len(C)+1):
        model = 'NH3_%s-%s'%(i,i)
        y = spectra.specmod(v,model,T_bg,Switch,para_type,*para)
        plt.plot(v,y,linewidth=2,c=C[i-1],label=model)
        
    plt.legend()
    plt.grid()
    plt.xlabel('v_lsr (km/s)')
    plt.ylabel('Intensity (Jy/sr)')

    
    I_max = np.full(len(C),np.nan)
    for i in range(1,len(C)+1):
        model = 'NH3_%s-%s'%(i,i)
        y = spectra.specmod(v,model,T_bg,Switch,para_type,*para)
        I_max[i-1] = y.max()
    I_max /= I_max.max()

    plt.subplot(122)
    plt.ylim(0,1.1)
    plt.scatter(np.arange(len(C)),I_max,color=C[:4],s=400)
    plt.plot(np.arange(len(C)),I_max,color='k')
    plt.grid()
    plt.xlabel('Transitions')
    plt.ylabel('Rel. max intensity (Jy/sr)')

interact(f,lgN=(14.,19.,.1),T_ex=(-3,200,1),sigma_v=(.1,2,.01))

interactive(children=(IntSlider(value=98, description='T_ex', max=200, min=-3), FloatSlider(value=16.5, descri…

<function __main__.f(T_ex, lgN, sigma_v)>

## func vs func_tau (needs modification)

In [ ]:
reload(spectra)

# parameters
T_bg = 2.73 # [K]
model = 'N2H+_1-0'# 'C17O_2-1' 'H13CO+_1-0' 'C18O_1-0' 'N2H+_1-0' 'NH2D_1-1'  'H13CO+_1-0' 
Switch_dict = {
    'thin': False,
    'intensity': True,
    'hyperfine': False,
    'collapsed_hyperfine': False,
}
v = np.linspace(-10,10,1000) # [km/s]
tex = 19.1
lgN = 15.65
v0 = 0 
sigma_v = .5

tau = spectra.calc_tau(tex,lgN,sigma_v,model)


# modeled spectra
para = [tex,lgN,v0,sigma_v]
y = spectra.func(v,model,T_bg,Switch_dict,*para)
para_t = [tex,tau,v0,sigma_v]
y_t = spectra.func_tau(v,model,T_bg,Switch_dict,*para_t)


# plt.figure(figsize=(10,8))
# plt.plot(v,y,label='y')
# plt.plot(v,y_t,label='y_t')
# plt.legend()
# plt.grid()
# plt.xlabel('v_lsr (km/s)')
# plt.ylabel('Intensity (Jy/sr)')
# plt.title(model+', tau=%.1f'%tau) 
# plt.show()


plt.figure(figsize=(10,8))
plt.plot(v,y_t/y)
plt.grid()
plt.xlabel('v_lsr (km/s)')
plt.title(model+', tau=%.1f'%tau) 
plt.show()